In [1]:
##from pyspark import SparkContext
##sc =SparkContext("local[*]")

In [2]:
sc.master

u'local[*]'

In [3]:
global Path    
if sc.master[0:5]=="local" :
   Path="file:/home/hduser/pythonwork/PythonProject/"
else:   
   Path="hdfs://master:9000/user/hduser/"
#如果您要在cluster模式執行(hadoop yarn 或Spark Stand alone)，請依照書上說明，先上傳檔案至HDFS目錄

In [4]:
print("開始匯入資料...")
rawDataWithHeader = sc.textFile(Path+"bank.csv")
rawDataWithHeader.take(2)

開始匯入資料...


[u'X1,X2, X3,Y', u'-48.2,6.8,1.6,0']

In [5]:
header = rawDataWithHeader.first() 
rData = rawDataWithHeader.filter(lambda x:x !=header)  
lines = rData.map(lambda x: x.split(","))
print("共計：" + str(lines.count()) + "筆")

共計：30筆


In [6]:
lines.collect()

[[u'-48.2', u'6.8', u'1.6', u'0'],
 [u'-49.2', u'-17.2', u'0.3', u'0'],
 [u'-19.2', u'-36.7', u'0.8', u'0'],
 [u'-18.1', u'-6.5', u'0.9', u'0'],
 [u'-98', u'-20.8', u'1.7', u'0'],
 [u'-129', u'-14.2', u'1.3', u'0'],
 [u'-4', u'-15.8', u'2.1', u'0'],
 [u'-8.7', u'-36.3', u'2.8', u'0'],
 [u'-59.2', u'-12.8', u'2.1', u'0'],
 [u'-13.1', u'-17.6', u'0.9', u'0'],
 [u'-38', u'1.6', u'1.2', u'0'],
 [u'-57.9', u'0.7', u'0.8', u'0'],
 [u'-8.8', u'-9.1', u'0.9', u'0'],
 [u'-64.7', u'-4', u'0.1', u'0'],
 [u'-11.4', u'4.8', u'0.9', u'0'],
 [u'43', u'16.4', u'1.3', u'1'],
 [u'47', u'16', u'1.9', u'1'],
 [u'-3.3', u'4', u'2.7', u'1'],
 [u'35', u'20.8', u'1.9', u'1'],
 [u'46.7', u'12.6', u'0.9', u'1'],
 [u'20.8', u'12.5', u'2.4', u'1'],
 [u'33', u'23.6', u'1.5', u'1'],
 [u'26.1', u'10.4', u'2.1', u'1'],
 [u'68.6', u'13.8', u'1.6', u'1'],
 [u'37.3', u'33.4', u'3.5', u'1'],
 [u'59', u'23.1', u'5.5', u'1'],
 [u'49.6', u'23.8', u'1.9', u'1'],
 [u'12.5', u'7', u'1.8', u'1'],
 [u'37.3', u'34.1', u'1.5', u'1

In [7]:
import numpy as np
def extract_features(field,featureEnd):
    #擷取分類特徵欄位
       #第3个欄位不是特徵欄位
    numericalFeatures=[float(id)  for  id in field[0:2]]    
    #回傳「分類特徵欄位」
    return numericalFeatures

In [8]:
def extract_label(field):
    label=(field[-1])
    return float(label)

In [13]:
labelRDD = lines.map( lambda r: extract_label(r))
print labelRDD.collect()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [14]:
from pyspark.mllib.regression import LabeledPoint
labelpointRDD = lines.map( lambda r: 
     LabeledPoint(
                extract_label(r),
                extract_features(r,len(r) - 1)))

In [15]:
labelpointRDD.take(2)
trainData=labelpointRDD
trainData.persist()

PythonRDD[11] at RDD at PythonRDD.scala:48

In [16]:
from pyspark.mllib.tree import DecisionTree
model=DecisionTree.trainClassifier( \
        trainData, numClasses=2, categoricalFeaturesInfo={}, \
        impurity="entropy", maxDepth=5, maxBins=5)

In [18]:
predictResult = model.predict([-25,2.5,0.5])  ###(X1,X2,X3) = [-25,2.5,0.5]

In [19]:
predictResult

0.0